In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from gpflow.kernels import Matern32, Kernel
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt
from datetime import datetime, timedelta
import ruptures as rpt
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Define the custom ChangePointKernel for Gaussian Process Regression
class ChangePointKernel(Kernel):
    def __init__(self, base_kernel, changepoints):
        super().__init__()
        self.base_kernel = base_kernel
        # changepoints: list of indices where a regime change is detected.
        self.changepoints = changepoints

    def get_region(self, X):
        cp = tf.constant(self.changepoints, dtype=X.dtype)
        regions = tf.searchsorted(cp, X[:, 0], side='right')
        return regions

    def K(self, X, X2=None):
        if X2 is None:
            X2 = X
        regions_X = self.get_region(X)
        regions_X2 = self.get_region(X2)
        regions_equal = tf.cast(tf.equal(tf.expand_dims(regions_X, 1), tf.expand_dims(regions_X2, 0)), X.dtype)
        base_cov = self.base_kernel.K(X, X2)
        return base_cov * regions_equal

    def K_diag(self, X):
        return self.base_kernel.K_diag(X)

def fetch_and_process_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        raise ValueError(f"No data found for {ticker} between {start_date} and {end_date}")
    # Prefer "Adj Close" if available, otherwise "Close"
    if "Adj Close" in data.columns:
        price_series = data["Adj Close"].copy()
    else:
        price_series = data["Close"].copy()
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    price_series.name = ticker
    returns = price_series.pct_change().dropna()
    scaler = StandardScaler()
    std_returns = scaler.fit_transform(returns.values.reshape(-1, 1))
    valid_index = returns.index
    return price_series.loc[valid_index], pd.Series(
        std_returns.flatten(), index=valid_index, name="Standardized Returns"
    )

def detect_changepoints(returns, lookback_window):
    # Use ruptures to detect changepoints using an RBF model.
    algo = rpt.Pelt(model="rbf").fit(returns.values)
    # Use the lookback_window as penalty; this is a simplified proxy for CPD LBW.
    result = algo.predict(pen=lookback_window)
    return result

def generate_signals(returns, changepoints, train_ratio, epochs, 
                     dropout_rate, lstm_hidden_units, learning_rate, batch_size):
    # Use time index as a feature.
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    # Use Gaussian Process Regression with the ChangePointKernel for trend extraction.
    base_kernel = Matern32()
    kernel = ChangePointKernel(base_kernel, changepoints)
    gpr = GPR(data=(X, y), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    trend = gpr.predict_f(X)[0].numpy().flatten()
    # Combine returns and trend to form a feature set.
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    split_index = int(len(features) * train_ratio)
    train_features = features[:split_index]
    train_labels = returns.values[:split_index]
    # Reshape features for LSTM input.
    train_features = train_features.reshape((train_features.shape[0], 1, train_features.shape[1]))
    # Build the LSTM model.
    model = Sequential([
        LSTM(lstm_hidden_units, return_sequences=True,
             input_shape=(train_features.shape[1], train_features.shape[2])),
        Dropout(dropout_rate),
        LSTM(int(lstm_hidden_units / 2)),
        Dense(1, activation="tanh")
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
    model.fit(train_features, train_labels, epochs=epochs, batch_size=batch_size, verbose=0)
    # Predict signals for all features.
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features, verbose=0).flatten()
    positions = np.clip(predicted_signals, -1, 1)
    return positions

def backtest_strategy(price_series, positions):
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    positions = pd.Series(positions, index=price_series.index, name=price_series.name)
    entries = positions > 0
    exits = positions < 0
    pf = vbt.Portfolio.from_signals(
        close=price_series,
        entries=entries,
        exits=exits,
        size=np.abs(positions),
        freq="1D"
    )
    return pf

def compute_trade_statistics(pf, price_series):
    # Extract trade records from the portfolio
    trades = pf.trades.records
    if trades.empty:
        win_rate = np.nan
        avg_win = np.nan
        avg_loss = np.nan
    else:
        # Assuming 'pnl' column exists representing profit (can be negative).
        wins = trades[trades.pnl > 0]
        losses = trades[trades.pnl < 0]
        win_rate = len(wins) / len(trades) if len(trades) > 0 else np.nan
        avg_win = wins.pnl.mean() if not wins.empty else np.nan
        avg_loss = losses.pnl.mean() if not losses.empty else np.nan
    return win_rate, avg_win, avg_loss

def main():
    # Fixed hyperparameters.
    params = {
        'batch_size': 128,
        'dropout_rate': 0.2,
        'learning_rate': 0.001,
        'lookback_window': 21,
        'lstm_hidden_units': 40,
        'epochs': 50,
        'train_ratio': 0.8
    }
    
    # List of top 10 cryptocurrencies (excluding stablecoins).
    top_cryptos = [
        "BTC-USD",  # Bitcoin
        "ETH-USD",  # Ethereum
        "BNB-USD",  # Binance Coin
        "XRP-USD",  # XRP
        "ADA-USD",  # Cardano
        "SOL-USD",  # Solana
        "DOT-USD",  # Polkadot
        "DOGE-USD", # Dogecoin
        "AVAX-USD", # Avalanche
        "MATIC-USD" # Polygon (MATIC)
    ]

    # Define test periods.
    periods = []
    for year in range(2019, 2025):
        period_label = str(year)
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        periods.append((period_label, start_date, end_date))
    # YTD 2025 using current date as provided in session (2025-02-24).
    periods.append(("2025_YTD", "2025-01-01", "2025-02-24"))

    results = []

    # Loop over each cryptocurrency and each period.
    for crypto in top_cryptos:
        for period_label, start_date, end_date in periods:
            print(f"Processing {crypto} for period {period_label} ({start_date} to {end_date})")
            try:
                price_series, returns = fetch_and_process_data(crypto, start_date, end_date)
                if len(returns) < params['lookback_window']:
                    print(f"Not enough data for {crypto} in period {period_label}. Skipping.")
                    continue
                
                changepoints = detect_changepoints(returns, lookback_window=params['lookback_window'])
                positions = generate_signals(
                    returns, changepoints,
                    train_ratio=params['train_ratio'],
                    epochs=params['epochs'],
                    dropout_rate=params['dropout_rate'],
                    lstm_hidden_units=params['lstm_hidden_units'],
                    learning_rate=params['learning_rate'],
                    batch_size=params['batch_size']
                )
                portfolio = backtest_strategy(price_series, positions)
                
                result = {
					"crypto": crypto,
					**{key: str(value) if pd.isna(value) else value for key, value in portfolio.stats().items()}
				}
                # result = {
                #     "crypto": crypto,
                #     "period": period_label,
                #     "start_date": start_date,
                #     "end_date": end_date,
                #     "total_return": total_return,
                #     "sharpe_ratio": sharpe_ratio,
                #     "max_drawdown": max_drawdown,
                #     "benchmark_return": benchmark_return,
                #     "win_rate": win_rate,
                #     "avg_win": avg_win,
                #     "avg_loss": avg_loss,
                #     "calmar_ratio": calmar_ratio,
                #     "omega_ratio": omega_ratio,
                #     "sortino_ratio": sortino_ratio,
                #     "portfolio_stats_file": stats_csv
                # }
                
                results.append(result)
                
                # stats_dict = portfolio.stats().apply(str).to_dict()
                # print(stats_dict)
            
            except Exception as e:
                print(f"Error processing {crypto} for period {period_label}: {e}")

    # Save all summary results into a DataFrame.
    results_df = pd.DataFrame(results)
    print("\nBacktest Results:")
    print(results_df)
    # Optionally, save the DataFrame to a CSV file.
    results_df.to_csv("Backtested/crypto_backtest_results.csv", index=False)

if __name__ == "__main__":
    main()

Processing BTC-USD for period 2019 (2019-01-01 to 2019-12-31)
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed

Processing BTC-USD for period 2020 (2020-01-01 to 2020-12-31)



/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed

Processing BTC-USD for period 2021 (2021-01-01 to 2021-12-31)


...

Processing MATIC-USD for period 2024 (2024-01-01 to 2024-12-31)



/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed

Processing MATIC-USD for period 2025_YTD (2025-01-01 to 2025-02-24)



/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Backtest Results:
       crypto      Start        End   Period  Start Value   End Value  \
0     BTC-USD 2019-01-02 2019-12-30 363 days        100.0  220.000637   
1     BTC-USD 2020-01-02 2020-12-30 364 days        100.0  248.180825   
2     BTC-USD 2021-01-02 2021-12-30 363 days        100.0  102.042283   
3     BTC-USD 2022-01-02 2022-12-30 363 days        100.0   86.240674   
4     BTC-USD 2023-01-02 2023-12-30 363 days        100.0  214.810635   
..        ...        ...        ...      ...          ...         ...   
62  MATIC-USD 2021-01-02 2021-12-30 363 days        100.0  100.320197   
63  MATIC-USD 2022-01-02 2022-12-30 363 days        100.0   99.347393   
64  MATIC-USD 2023-01-02 2023-12-30 363 days        100.0  100.212699   
65  MATIC-USD 2024-01-02 2024-12-30 364 days        100.0   99.865989   
66  MATIC-USD 2025-01-02 2025-02-23  53 days        100.0   99.990729   

    Total Return [%]  Benchmark Return [%]  Max Gross Exposure [%]  \
0         120.000637             8

In [2]:
df = pd.read_csv("Backtested/crypto_backtest_results.csv")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df

,crypto,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,Total Closed Trades,Total Open Trades,Open Trade PnL,Win Rate [%],Best Trade [%],Worst Trade [%],Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio
0,BTC-USD,2019-01-02,2019-12-30,363 days,100.0,220.000637,120.000637,84.941362,100.000000,0.0,21.784298,89 days,80,80,0,0.000000,37.500000,23.522550,-13.009973,6.693874,-1.794935,3 days 08:00:00,1 days 04:48:00,1.783945,1.500008,1.906125,5.552551,1.601877,3.289908
1,BTC-USD,2020-01-02,2020-12-30,364 days,100.0,248.180825,148.180825,312.870605,100.000000,0.0,27.196399,95 days,94,93,1,15.111893,43.010753,24.854946,-7.006916,4.760283,-1.779563,2 days 16:12:00,1 days 03:10:11.320754716,2.134234,1.430849,2.291780,5.471363,1.749915,4.837602
2,BTC-USD,2021-01-02,2021-12-30,363 days,100.0,102.042283,2.042283,46.847611,100.000000,0.0,46.281069,312 days,93,92,1,0.000000,34.782609,29.633437,-13.321745,5.479477,-2.528059,2 days 16:30:00,1 days 08:00:00,1.013095,0.022199,0.279548,0.044373,1.059252,0.394819
3,BTC-USD,2022-01-02,2022-12-30,363 days,100.0,86.240674,-13.759326,-64.932920,100.000000,0.0,36.758625,276 days,83,82,1,-0.206508,39.024390,17.253614,-9.707194,3.294533,-2.278779,3 days 12:00:00,1 days 10:33:36,0.883991,-0.165278,-0.168197,-0.376228,0.965211,-0.239499
4,BTC-USD,2023-01-02,2023-12-30,363 days,100.0,214.810635,114.810635,152.610938,100.000000,0.0,12.008129,114 days,89,89,0,0.000000,41.573034,22.173204,-5.180921,4.287927,-1.380787,2 days 12:19:27.567567567,1 days 01:50:46.153846153,1.957142,1.290007,2.446876,9.636593,1.853984,5.545457
5,BTC-USD,2024-01-02,2024-12-30,364 days,100.0,155.179572,55.179572,106.066274,100.000000,0.0,21.804052,211 days,92,92,0,0.000000,35.869565,26.811372,-6.032106,4.640142,-1.683838,3 days 02:10:54.545454545,1 days 02:50:50.847457627,1.414008,0.599778,1.445287,2.539299,1.373461,2.590598
6,BTC-USD,2025-01-02,2025-02-23,53 days,100.0,98.861239,-1.138761,-0.632648,100.000000,0.0,11.285459,33 days,11,10,1,-0.312001,40.000000,6.745574,-3.484628,2.249992,-1.583070,4 days 18:00:00,2 days 08:00:00,0.916735,-0.082676,-0.073946,-0.672048,0.986506,-0.108692
7,ETH-USD,2019-01-02,2019-12-30,363 days,100.0,112.161795,12.161795,-14.456327,100.000000,0.0,19.283130,187 days,90,90,0,0.000000,34.444444,23.834996,-8.880256,6.467234,-2.014433,3 days 02:19:21.290322580,1 days 06:54:54.915254237,1.122995,0.135131,0.485636,0.634375,1.136467,0.764348
8,ETH-USD,2020-01-02,2020-12-30,364 days,100.0,340.386784,240.386784,489.920650,100.000000,0.0,19.092471,74 days,86,85,1,63.407341,48.235294,29.239038,-8.055328,6.270061,-2.350039,2 days 20:29:16.097560975,1 days 04:21:49.090909090,2.329053,2.082111,2.969929,12.650754,1.987324,5.719183
9,ETH-USD,2021-01-02,2021-12-30,363 days,100.0,266.847340,166.847340,379.494430,100.000000,0.0,39.598864,202 days,91,90,1,0.000000,41.111111,58.055761,-15.933166,8.165990,-3.167163,3 days 00:38:55.135135135,1 days 05:26:02.264150943,1.443348,1.853859,1.809076,4.249978,1.470087,3.117629
